## Branch

The `Session` object now supports working on multiple branches. By default, each session begins with a `'main'` branch.

In [1]:
import lionagi as li

In [2]:
system = "you are asked to perform as a function picker and parameter provider"
session = li.Session(system=system)
session.branches

{'main': <lionagi.core.branch.Branch at 0x13748a080>}

### `new_branch`

Create a new branch by passing:

- the name of the new branch 
- the name of the branch to clone from.

In [3]:
session.new_branch('branch1', 'main')
session.branches

{'main': <lionagi.core.branch.Branch at 0x13748a080>,
 'branch1': <lionagi.core.branch.Branch at 0x13748a6e0>}

### `switch_branch`

There is always one current active branch. When calling methods in the `session` object without specifying a branch, they will be default to the `current_branch`.

To switch the current branch, provide the name of the branch you want to switch to.

In [4]:
session.current_branch_name, session.current_branch

('main', <lionagi.core.branch.Branch at 0x13748a080>)

In [5]:
session.switch_branch('branch1')
session.current_branch_name, session.current_branch

('branch1', <lionagi.core.branch.Branch at 0x13748a6e0>)

For the `current_branch`, most branch methods can be called directly. Otherwise, the branch needs to be specified.

For example, let's say we want to add the 'multiply' tool to the 'main' branch and the 'add' tool to the 'branch1' branch.

In [6]:
tool_1=[
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Perform multiplication on two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "number1": {
                        "type": "number",
                        "description": "a number to multiply, e.g. 5.34",
                    },
                    "number2": {
                        "type": "number",
                        "description": "a number to multiply, e.g. 17",
                    },
                },
                # specify which parameters are required for the model to respond when function calling
                "required": ["number1", "number2"],
            },
        }
    }
]

def multiply(number1, number2):
    return number1*number2

# created a tool object
tool_mul = li.Tool(func=multiply, schema_=tool_1[0])

In [7]:
tool_2=[
    {
        "type": "function",
        "function": {
            "name": "add",
            "description": "Perform addition on two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "number1": {
                        "type": "number",
                        "description": "a number to add, e.g. 5.34",
                    },
                    "number2": {
                        "type": "number",
                        "description": "a number to add, e.g. 17",
                    },
                },
                # specify which parameters are required for the model to respond when function calling
                "required": ["number1", "number2"],
            },
        }
    }
]

def add(number1, number2):
    return number1+number2

tool_add = li.Tool(func=add, schema_=tool_2[0])

In [8]:
# register tool_mul to 'main' branch
session.branches['main'].register_tools(tool_mul)

# register tool_add to 'branch1' branch
# since 'branch1' is the current branch, we may use the shortcut
session.register_tools(tool_add)

In [9]:
# if you check the tool registry, they should be there now
session.branches['main'].tool_manager.registry

{'multiply': Tool({"node_id":"6e229c5e2e53790d523a6ce2a67ae5c1","metadata":{},"label":null,"related_nodes":[],"content":null,"func":"multiply","parser":null,"schema_":{"type":"function","function":{"name":"multiply","description":"Perform multiplication on two numbers","parameters":{"type":"object","properties":{"number1":{"type":"number","description":"a number to multiply, e.g. 5.34"},"number2":{"type":"number","description":"a number to multiply, e.g. 17"}},"required":["number1","number2"]}}}})}

In [10]:
session.branches['branch1'].tool_manager.registry

{'add': Tool({"node_id":"36917d0efc1f1c73b6c9a6cfe75c7e02","metadata":{},"label":null,"related_nodes":[],"content":null,"func":"add","parser":null,"schema_":{"type":"function","function":{"name":"add","description":"Perform addition on two numbers","parameters":{"type":"object","properties":{"number1":{"type":"number","description":"a number to add, e.g. 5.34"},"number2":{"type":"number","description":"a number to add, e.g. 17"}},"required":["number1","number2"]}}}})}

When calling `Session`'s core methods, if the branch is not specified, they default to the `current_branch`.

In [11]:
task = "Think step by step, understand the following basic math question and provide parameters for function calling."

# when using respond_mode as json to enforce output format, you need to provide specifying details in instruction
json_format = {"number1": "x", "number2": "y"}

instruct = {"Task": task, "json_format": json_format}

In [12]:
question = "There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?"
question2 = "There are three fruits in total, each with 2 different colors, how many unique kinds of fruits are there?"

context1 = {"Question1": question, "question2": question2}

question3 = "There are two cards facing up and three facing down, how many cards are there in total?"
context2 = {"Question3": question3}

In [13]:
session.llmconfig.update({
    "temperature":0.35,
    "tool_choice": "auto", 
    "response_format": {'type':'json_object'}
})

In [14]:
response_main = await session.initiate(instruction=instruct, 
                                       context=context1, 
                                       branch='main', 
                                       tools=True)

In [15]:
# since the the current branch is branch1 in this case, 
# we do not need to specify the branch
response_branch1 = await session.initiate(instruction=instruct, 
                                          context=context2, 
                                          tools=True)

In [16]:
for content in session.branches['main'].messages['content']:
    print(content)

{"system_info": "you are asked to perform as a function picker and parameter provider"}
{"instruction": {"Task": "Think step by step, understand the following basic math question and provide parameters for function calling.", "json_format": {"number1": "x", "number2": "y"}}, "context": {"Question1": "There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?", "question2": "There are three fruits in total, each with 2 different colors, how many unique kinds of fruits are there?"}}
{"action_list": [{"recipient_name": "functions.multiply", "parameters": {"number1": 3, "number2": 4}}, {"recipient_name": "functions.multiply", "parameters": {"number1": 3, "number2": 2}}]}
{"action_response": {"function": "multiply", "arguments": {"number1": 3, "number2": 4}, "output": 12}}
{"action_response": {"function": "multiply", "arguments": {"number1": 3, "number2": 2}, "output": 6}}


In [17]:
for content in session.current_branch.messages['content']:
    print(content)

{"system_info": "you are asked to perform as a function picker and parameter provider"}
{"instruction": {"Task": "Think step by step, understand the following basic math question and provide parameters for function calling.", "json_format": {"number1": "x", "number2": "y"}}, "context": {"Question3": "There are two cards facing up and three facing down, how many cards are there in total?"}}
{"action_list": [{"action": "action_add", "arguments": "{\"number1\":2,\"number2\":3}"}]}
{"action_response": {"function": "add", "arguments": {"number1": 2, "number2": 3}, "output": 5}}


### `merge_branch`

we can merge one branch into another branch by passing:

- from_: The name of the branch to merge from.
- to_: The name of the branch to merge into.
- update: Specifies whether to update the target branch with the source branch's data when conflicts occur. Default to True.
- if_delete: Indicates whether to delete the source branch after merging. If the source branch is the `current_branch`, the `current_branch` will switch to the merge target branch. Default to False.

we are using `pandas.Dataframe`'s outer merge logic for `merge_branch`.

In [18]:
session.merge_branch(from_='branch1', to_='main')

In [19]:
session.branches['main'].messages

,node_id,role,name,timestamp,content
0,90423e5f52828fdc399f3cb6c2c0e005,system,system,2024-01-18 23:01:24.187353,"{""system_info"": ""you are asked to perform as a..."
1,e987c8b6759d9e2e66dd0f790eb4fd06,user,user,2024-01-18 23:01:24.285710,"{""instruction"": {""Task"": ""Think step by step, ..."
2,2a5402aad82e5d3af77fbb8e4054ae22,assistant,action_request,2024-01-18 23:01:35.077627,"{""action_list"": [{""recipient_name"": ""functions..."
3,18311dbec86494b5fb4278cad8069683,assistant,action_response,2024-01-18 23:01:35.082164,"{""action_response"": {""function"": ""multiply"", ""..."
4,8ba3374a6cbd0e9351f076bff8cfa668,assistant,action_response,2024-01-18 23:01:35.083547,"{""action_response"": {""function"": ""multiply"", ""..."
5,574f04ca96825a92e1c70f334e951d5e,user,user,2024-01-18 23:01:35.095243,"{""instruction"": {""Task"": ""Think step by step, ..."
6,7cc96ffa542cd3ce5e63f40938194cca,assistant,action_request,2024-01-18 23:01:37.661174,"{""action_list"": [{""action"": ""action_add"", ""arg..."
7,f1d115f908b244b5dabbf37509b9d220,assistant,action_response,2024-01-18 23:01:37.663784,"{""action_response"": {""function"": ""add"", ""argum..."


In [20]:
for content in session.branches['main'].messages['content']:
    print(content)

{"system_info": "you are asked to perform as a function picker and parameter provider"}
{"instruction": {"Task": "Think step by step, understand the following basic math question and provide parameters for function calling.", "json_format": {"number1": "x", "number2": "y"}}, "context": {"Question1": "There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?", "question2": "There are three fruits in total, each with 2 different colors, how many unique kinds of fruits are there?"}}
{"action_list": [{"recipient_name": "functions.multiply", "parameters": {"number1": 3, "number2": 4}}, {"recipient_name": "functions.multiply", "parameters": {"number1": 3, "number2": 2}}]}
{"action_response": {"function": "multiply", "arguments": {"number1": 3, "number2": 4}, "output": 12}}
{"action_response": {"function": "multiply", "arguments": {"number1": 3, "number2": 2}, "output": 6}}
{"ins

### `delete_branch`

To delete a branch, provide the name of the branch you want to delete.

NOTE: `current_branch` cannot be deleted. If you want to delete it, please switch to another branch first.

In [21]:
# Suppose we want to the delete 'branch1', let's switch to 'main' first 
# since 'branch1' is the current_branch now.
session.switch_branch('main')
session.current_branch_name

'main'

In [22]:
session.delete_branch('branch1')

Branch branch1 is deleted.


True